# Árvores II - Tarefa 2

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix

### 1. Carregar as bases

Vamos carregar as bases lidas na tarefa passada. Se você salvou essas bases em arquivo texto, basta fazer a leitura com o comando ```pd.read_csv``` das seguintes bases:

- X_train
- Y_train
- X_test
- Y_test

Não se esqueça de considerar a leitura dos índices dos arquivos no ```read_csv()```!

In [14]:
# Carregando os dados de treino e teste - X variáveis explicativas, y variável resposta

X_train = pd.read_csv("X_train.csv")
y_train = pd.read_csv(r'arquivos\UCI HAR Dataset\UCI HAR Dataset\train\y_train.txt', header=None, delim_whitespace=True)
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv(r'arquivos\UCI HAR Dataset\UCI HAR Dataset\test\y_test.txt', header=None, delim_whitespace=True)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(7352, 563)
(7352, 1)
(2947, 563)
(2947, 1)


#### Deletando a coluna Unnamed: 1 que foi criada ao carregar o arquivo, isso aconteceu por causa do uso dos indices no exercício anterior

In [24]:
X_train = X_train.drop(columns=['Unnamed: 1'])
X_train.head(3)

,subject_train,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,1.0,0.288585,-0.020294,-0.132905,-0.995279,-0.983111,-0.913526,-0.995112,-0.983185,-0.923527,...,-0.074323,-0.298676,-0.710304,-0.112754,0.030400,-0.464761,-0.018446,-0.841247,0.179941,-0.058627
1,1.0,0.278419,-0.016411,-0.123520,-0.998245,-0.975300,-0.960322,-0.998807,-0.974914,-0.957686,...,0.158075,-0.595051,-0.861499,0.053477,-0.007435,-0.732626,0.703511,-0.844788,0.180289,-0.054317
2,1.0,0.279653,-0.019467,-0.113462,-0.995380,-0.967187,-0.978944,-0.996520,-0.963668,-0.977469,...,0.414503,-0.390748,-0.760104,-0.118559,0.177899,0.100699,0.808529,-0.848933,0.180637,-0.049118


In [26]:
X_test = X_test.drop(columns=['Unnamed: 1'])
X_test.head(3)

,subject_test,tBodyAcc-mean()-X,tBodyAcc-mean()-Y,tBodyAcc-mean()-Z,tBodyAcc-std()-X,tBodyAcc-std()-Y,tBodyAcc-std()-Z,tBodyAcc-mad()-X,tBodyAcc-mad()-Y,tBodyAcc-mad()-Z,...,fBodyBodyGyroJerkMag-meanFreq(),fBodyBodyGyroJerkMag-skewness(),fBodyBodyGyroJerkMag-kurtosis(),"angle(tBodyAccMean,gravity)","angle(tBodyAccJerkMean),gravityMean)","angle(tBodyGyroMean,gravityMean)","angle(tBodyGyroJerkMean,gravityMean)","angle(X,gravityMean)","angle(Y,gravityMean)","angle(Z,gravityMean)"
0,2.0,0.257178,-0.023285,-0.014654,-0.938404,-0.920091,-0.667683,-0.952501,-0.925249,-0.674302,...,0.071645,-0.330370,-0.705974,0.006462,0.162920,-0.825886,0.271151,-0.720009,0.276801,-0.057978
1,2.0,0.286027,-0.013163,-0.119083,-0.975415,-0.967458,-0.944958,-0.986799,-0.968401,-0.945823,...,-0.401189,-0.121845,-0.594944,-0.083495,0.017500,-0.434375,0.920593,-0.698091,0.281343,-0.083898
2,2.0,0.275485,-0.026050,-0.118152,-0.993819,-0.969926,-0.962748,-0.994403,-0.970735,-0.963483,...,0.062891,-0.190422,-0.640736,-0.034956,0.202302,0.064103,0.145068,-0.702771,0.280083,-0.079346


### 2. Divisão da base em Treino, Validação e Teste

A base já se encontra dividida em Treino e Validação. O que vamos fazer então é extrair uma base de Validação da base de Treino.

Extraia 25% da base de treino como base de validação.

In [33]:
# Dividindo em treino e teste uma segunda vez, agora usando os valores de teste para alimentar essa nova base
# Sendo assim, os valores de validação vão ser ainda menores mais ou menos desse tamanho:
#736,75 -> test
#2210,25 -> treino
X_train_val, X_test_val, y_train_val, y_test_val = train_test_split(X_test, y_test, test_size=.25, random_state=2)
print(X_train_val.shape)
print(X_test_val.shape)

(2210, 562)
(737, 562)


### 3. Melhores 3 variáveis

Rode uma árvore com profundidade máxima igual a 4 para prever a atividade humana com todas as variáveis.
Observe a importância das variáveis e considere as 3 variáveis com maior importância para os próximos passos.
Dica: utilize o atributo ```clf.feature_importances_``` da árvore treinada.

In [45]:
# Criando a árvore com profundidade 4
tree = DecisionTreeClassifier(random_state=1, max_depth=4)
tree.fit(X_train, y_train)
# Como é a primeira vez que uso o feature_importances_ achei melhor visualizar como se comporta
importances = tree.feature_importances_
importances

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.28379363, 0.        ,
       0.        , 0.        , 0.        , 0.01158699, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [46]:
# Como só tinha os valores das importâncias em um array, transformei em DF e usei o Feature para associar coluna e importância
importancias_df = pd.DataFrame({'Feature': X_train_val.columns, 'Importance': importances})
importancias_df

,Feature,Importance
0,subject_test,0.00000
1,tBodyAcc-mean()-X,0.00000
2,tBodyAcc-mean()-Y,0.00000
3,tBodyAcc-mean()-Z,0.00000
4,tBodyAcc-std()-X,0.00000
...,...,...
557,"angle(tBodyGyroMean,gravityMean)",0.00000
558,"angle(tBodyGyroJerkMean,gravityMean)",0.00000
559,"angle(X,gravityMean)",0.00000
560,"angle(Y,gravityMean)",0.16645


In [47]:
# Ordenando e mostrando as 3 primeiras linhas, que contem as 3 variáveis mais importântes
importancias_df = importancias_df.sort_values(by='Importance', ascending=False)
importancias_df.head(3)

,Feature,Importance
53,tGravityAcc-min()-X,0.283794
394,"fBodyAccJerk-bandsEnergy()-1,24",0.255008
560,"angle(Y,gravityMean)",0.166450


In [48]:
# Criando uma lista das variáveis importantes para facilitar o uso
importantes = ['tGravityAcc-min()-X', 'fBodyAccJerk-bandsEnergy()-1,24', 'angle(Y,gravityMean)']

### 4. Construa uma árvore com as 3 melhores variáveis

Utilizando as três variáveis encontradas acima, construa uma árvore de decisão. Encontre o melhor ```ccp_alpha``` utilizando a base de validação, conforme a estrutura que vimos em aula.

### 5. Avaliação do modelo

Avalie a árvore encontrada no item anterior na base de testes.

Como o exercício 4 e 5 estão interligados, pois todo o ajuste foi feito com os dados de X_train_importante porém o score final foi calculado com o X_test_importante, então levei em consideração como se fosse apenas um exercício.

In [61]:
# Criando novo conjunto de dados com apenas as 3 variáveis mais importantes
X_train_importante = X_train_val[importantes]
X_test_importante = X_test_val[importantes]

# Ajustando os dados com o novo conjunto de dados e suas respostas
tree_importante = DecisionTreeClassifier(random_state=1, max_depth=4)
tree_importante.fit(X_train_importante, y_train_val)

# Calcula a complexidade dos caminhos e suas possiveis podas com alpha como no exercício da aula anterior
path = tree_importante.cost_complexity_pruning_path(X_train_importante, y_train_val)
ccp_alphas, inpurities = path.ccp_alphas, path.impurities
print("Valores alfa:", ccp_alphas)
print("Impurezas:", inpurities)

Valores alfa: [0.         0.00321374 0.00360828 0.01239125 0.02855214 0.0439299
 0.07736967 0.16944722 0.17325747]
Impurezas: [0.32020019 0.32341393 0.32702221 0.33941346 0.3679656  0.4118955
 0.48926517 0.65871239 0.83196986]


In [62]:
# Salvando a lista de árvores em tree_list com base nos ccp_alphas criados anteriormente
tree_list = []
for ccp_alpha in ccp_alphas:
    tree = DecisionTreeClassifier(random_state=1, ccp_alpha=ccp_alpha).fit(X_train_importante, y_train_val)
    tree_list.append(tree)

# Salvando cada valor de score na lista de treino e teste
train_scores = [tree.score(X_train_importante, y_train_val) for tree in tree_list] 
test_scores = [tree.score(X_test_importante, y_test_val) for tree in tree_list]

In [65]:
# Salvando o melhor ccp alpha obtido na base de teste
indice_melhor_ccp = test_scores.index(max(test_scores))
melhor_ccp_alpha = ccp_alphas[indice_melhor_ccp]
melhor_ccp_alpha

0.0